In [1]:
from MasterThesis.EDA import read_geotiff_image, read_numpy_image, less_cloudy_image
from MasterThesis.preprocessing import from_array_to_geotiff
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import clear_output
import numpy as np
import os
import sys
import elevation
import re
import shutil
from tqdm import tqdm
from MasterThesis import EDA
import multiprocessing as mp

/media/omar/storage/gdrive/Maestria/MasterThesis/MasterThesis/preprocessing.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
path_to_data = "/media/omar/storage/gdrive/Maestria/Datasets/GeoDataset" 

In [3]:
metadata = pd.read_csv("/media/omar/storage/gdrive/Maestria/Theses/Preprocessing/metadata_raw_images.csv")
metadata = pd.read_csv("/media/omar/storage/gdrive/Maestria/Datasets/GeoDataset/metadata_v2/metadata.csv", usecols=["Image"])
metadata.Image = metadata.Image.str.replace("/chip.npy", "", regex=False)
metadata.rename({"Image": "image"}, axis=1, inplace=True)
# patches = ['18NTN', '18NTP', '18NUL', '18NUM', '18NUN', '18NUP']
# metadata = metadata.query("patch in ['18NTN', '18NTP', '18NUL', '18NUM', '18NUN', '18NUP']")
# metadata = metadata.query("patch in ['18NTN', '18NTP', '18NUL']")
metadata

,image
0,"18NTN/18NTN_8_5/(0, 0)"
1,"18NTN/18NTN_8_5/(0, 100)"
2,"18NTN/18NTN_8_5/(0, 200)"
3,"18NTN/18NTN_8_5/(0, 300)"
4,"18NTN/18NTN_8_5/(0, 400)"
...,...
59725,"18PWQ/18PWQ_2_9/(900, 500)"
59726,"18PWQ/18PWQ_2_9/(900, 600)"
59727,"18PWQ/18PWQ_2_9/(900, 700)"
59728,"18PWQ/18PWQ_2_9/(900, 800)"


In [24]:
def get_elevation_map(path_to_metadata, path_to_save_map):

    corners = pd.read_pickle(path_to_metadata)["corners"]
    west, north = corners["nw"][::-1]
    east, south = corners["se"][::-1]

    elevation.clip(bounds=(west, south, east, north), output=f"{path_to_save_map}", margin="1%")
    elevation.clean()

    clear_output()

In [25]:
path_to_metadata = "/media/omar/storage/gdrive/Maestria/Datasets/GeoDataset/Dataset/18NVK/18NVK_0_1/(0, 0)/metadata.pkl"
path_to_save_map =  f"{path_to_data}/Elevations/elevation.tif"
get_elevation_map(path_to_metadata, path_to_save_map)

In [26]:
path = "/media/omar/storage/gdrive/Maestria/Datasets/GeoDataset/Dataset/18NTN/18NTN_8_5/(0, 400)/metadata.pkl"
save_path = f"{path_to_data}/Elevations/elevation.tif"
#get_elevation_map(path, save_path)

In [27]:
EDA.read_geotiff_image("/media/omar/storage/gdrive/Maestria/Datasets/GeoDataset/Elevations/elevation.tif").shape

(33, 33)

In [28]:
elevation.clip?

Signature: elevation.clip(bounds, output='out.tif', margin='0', **kwargs)
Docstring:
Clip the DEM to given bounds.

:param bounds: Output bounds in 'left bottom right top' order.
:param output: Path to output file. Existing files will be overwritten.
:param margin: Decimal degree margin added to the bounds. Use '%' for percent margin.
:param cache_dir: Root of the DEM cache folder.
:param product: DEM product choice.
File:      ~/.pyenv/versions/3.7.13/envs/MasterThesis/lib/python3.7/site-packages/elevation/datasource.py
Type:      function


In [ ]:
letters="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
letters_map=[{l:i for i, l in enumerate(letters)}][0]

In [ ]:
def generate_elevation_maps(metadata):

    metadata_elevations = []
    for path in tqdm(metadata.image):
        print(path)
        if not os.path.isdir(f"{path_to_data}/Elevations/{path}"):
            os.makedirs(f"{path_to_data}/Elevations/{path}")

        get_elevation_map(f"{path_to_data}/Dataset/{path}/metadata.pkl", f"{path_to_data}/Elevations/elevation.tif")

        if not os.path.isdir(f"{path_to_data}/sentinel_geo_images/{path}"):
            os.makedirs(f"{path_to_data}/sentinel_geo_images/{path}")

        #img = read_numpy_image(f"{path_to_data}/Dataset/{path}/chip.npy")
        #from_array_to_geotiff(f"{path_to_data}/sentinel_geo_images/{path}/chip.tif", img[:3], f"{path_to_data}/Dataset/{path}/metadata.pkl", crs=3116)

        shutil.move(f"{path_to_data}/Elevations/elevation.tif", f"{path_to_data}/Elevations/{path}/elevation.tif")

        id_number = re.sub(r"[a-zA-Z_\(\)\,\\\/\s+]", "", path)
        id_letter = re.sub(r"[\d_\(\)\,\\\/\s+]", "", path)
        id_letter = "".join(str(letters_map[l]) for l in id_letter[:3])
        id = int(id_number + id_letter)
        metadata_elevations.append({"Id":id, "Elevation":f"{path}/elevation.tif", "Image":f"{path}/chip.npy"})
        

    return pd.DataFrame(metadata_elevations)

In [ ]:
metadata_elevations = generate_elevation_maps(metadata)

In [ ]:
print(metadata_elevations.shape)
metadata_elevations.to_csv("/media/omar/storage/gdrive/Maestria/Datasets/GeoDataset/Elevations/metadata.csv",index=False)
metadata_elevations.head()

In [ ]:
i = 112

fig, ax = plt.subplots(1, 2, figsize = (10, 10))
ax[0].imshow(read_geotiff_image(f"{path_to_data}/Elevations/{metadata_elevations.Elevation[i]}"))
img = read_numpy_image(f"{path_to_data}/Dataset/{metadata_elevations.Image[i]}")
ax[1].imshow(np.clip(img[:3].transpose(1, 2, 0), 0, 6000) / 6000)


